In [1]:
import pandas as pd
import time
import datetime
import re
import glob
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import pickle
import json
from tools import *
from required_data_src import *

data/org_data\2018_Namelist&OM Report_procurement.xlsx
data/org_data\2019_Namelist&OM Report_procurement.xlsx
data/org_data\2020_Namelist&OM Report_procurement.xlsx
data/org_data\2021_Namelist&OM Report_procurement.xlsx
data/org_data\2022_Namelist&OM Report_procurement.xlsx
data\Turnover Report_Procurement_201812_Final.xlsx
data\Turnover Report_Procurement_201912_Final.xlsx
data\Turnover Report_Procurement_202012-final-exclude DGO.xlsx
data\Turnover Report_Procurement_202112.xlsx
data\Turnover Report_Procurement_202212.xlsx
data\Turnover Report_Procurement_202310 APAC.xlsx


In [20]:
# 薪资数据ID是有重复的，有人有多次变动
# duplicates = cr_src.groupby('ID')['Gender'].count()[cr_src.groupby('ID')['Gender'].count() > 1]

In [23]:
# cr_src = get_cr_src(runtime=2)
# org_data = get_org_data(runtime=2)
# turnover_data = get_turnover_data(runtime=2)

### Data preprocess
#### 离职数据

In [3]:
# turnover_data = get_turnover_data(runtime=2)
turnover_data['used_region_flag'] = turnover_data.apply(lambda x: define_used_region(x), axis=1)
turnover_data = turnover_data.query('(used_region_flag > 0) & (LeaveType == "Termination-Voluntary")')
turnover_data = turnover_data.query('ID != "52710485-744498"')     # onboardDate数据错误

turnover_data['OnboardDate'] = pd.to_datetime(turnover_data['OnboardDate'].astype(str))
turnover_data['LastWorkingdate'] = pd.to_datetime(turnover_data['LastWorkingdate'])
turnover_data['duration'] = (turnover_data['LastWorkingdate'] - turnover_data['OnboardDate']).dt.days   # 
turnover_data = turnover_data.query("~((PAID == 7109) & (Age == 35))")   # 有一个ID在离职表里是重复的，没有真的离职，从7109变成了CN21，而且还升职了
remove_id = movement_src.query("ReasonofChange == 'Repatriate-Return to Home'")['ID'].tolist()
turnover_data = turnover_data[~turnover_data['ID'].isin(remove_id)]     #这个情况也不是主动离职
turnover_data['Band_V'] = turnover_data['Band_V'].map(lambda x: re.sub(r'-A|-B','',x))

In [4]:
turnover_data['current_band'] = turnover_data['Band_V'].replace(roam_to_num_dict)      # 当前职级
# turnover_data['current_year'] = '2023-10-31'     # 截止到2023.10.31的时候已经不是fte了
turnover_data['current_year'] = turnover_data['LastWorkingdate'] - datetime.timedelta(days=31*6)
turnover_data['is_fte'] = 0                      # FTE数据标签（负样本）
turnover_data['on_duty_days'] = (turnover_data['current_year'] - turnover_data['OnboardDate']).dt.days    # 截止到半年前已经就职了多少天
turnover_data['current_year'] = turnover_data['current_year'].dt.strftime("%Y-%m-%d")

In [5]:
fte_used_columns = ['ID', 'Age', 'Gender', 'PAID', 'MacroEntity', 'positionName', 'OnboardDate', 'duration', 
                    'current_year', 'is_fte', 'current_band', 'on_duty_days']
turnover_used_columns = fte_used_columns + ['LastWorkingdate']
turnover_data = turnover_data[turnover_used_columns]

In [6]:
final_turnover_data = get_one_fte_snapshot(turnover_data)     # 获取movement的标签
for i, year in enumerate([2022, 2021, 2020, 2019, 2018]):
    turnover = turnover_data.copy()
    turnover['current_year'] = f'{year}-12-31'
    # LastWorkingdate 要大于 current + 6months
    current_lag_6_date = datetime.datetime(*time.strptime(f'{year}-12-31', '%Y-%m-%d')[:6]) + datetime.timedelta(days=31*6)
    current_lag_6_date_str = current_lag_6_date.strftime('%Y-%m-%d')
    turnover.query("(OnboardDate <= current_year) & (LastWorkingdate > @current_lag_6_date_str)")
    turnover['duration'] = (turnover['LastWorkingdate'] - turnover['OnboardDate']).dt.days
    turnover['on_duty_days'] = (pd.to_datetime(turnover['current_year']) - turnover['OnboardDate']).dt.days
    turnover['Age'] = turnover['Age'] - i - 1
    turnover['is_fte'] = 1
    turnover = get_one_fte_snapshot(turnover)
    final_turnover_data = pd.concat([final_turnover_data, turnover])
final_turnover_data['src_file'] = 'turnover'     # 标记这部分数据来自于turnover

#### 在职员工数据

In [8]:
# TODO 对于预测数据，是要把on_duty_days的计算时间改成10.31

In [220]:
fte = pd.read_excel('data/Turnover Report_Procurement_202310 APAC.xlsx', sheet_name='FTE') # 截止到2023年10月底的在职员工数据
fte['ID'] = fte['ID'].astype(str).str.split('.').apply(
            lambda x: "-".join([str(list(x)[0]), str(list(x)[1])[:6]]))

In [347]:
X_train.isnull().sum()

Age                               0
Gender                            0
on_duty_days                      0
move_up_times                     0
days_since_recent_move_up       287
move_down_times                   0
days_since_recent_demotion      387
other_move_times                  0
days_since_recent_other_move    205
end_of_date_band                  0
end_of_date_position              0
end_of_date_MacroEntity           0
end_of_date_paid                  0
year                              0
Employee Engagement Index       259
Manager Effectiveness Index     259
recent_cr                       108
days_since_recent_cr_change     108
cr_change_times                 108
recent_cr_diff                  167
cr_change_vs_term               108
Organizational Unit               0
Line Manager ID                   0
Job Family                        0
员工子组(OM)                          0
工作地 ID(OM)                        0
Functional Area                   0
dtype: int64

In [222]:
used_columns = item_def['FTE'].dropna().tolist()
fte = fte[used_columns]
fte.rename(columns={'LegalEntity': 'PAID', 'PositionName_EN':'positionName', 
                    'OnboardDate_Group': 'OnboardDate', 'PersBand':'Band_V',
                    'AGE':'Age', 'EmployeeID':'ID'}, inplace=True)
fte['used_region_flag'] = fte.apply(lambda x: define_used_region(x), axis=1)
fte = fte.query('used_region_flag > 0')
fte['OnboardDate'] = pd.to_datetime(fte['OnboardDate'])
fte['duration'] = (pd.to_datetime('2023-10-31') - fte['OnboardDate']).dt.days       # 半年后2023-10-31还在，按照半年前4月底做feature
fte['on_duty_days'] = (pd.to_datetime('2023-04-30') - fte['OnboardDate']).dt.days
# fte数据要每年做出来一波还在职的作为数据集
fte['current_year'] = '2023-04-30'
fte['is_fte'] = 1                      # FTE数据标签
fte['current_band'] = fte['Band_V'].replace(roam_to_num_dict)      # 当前职级

C:\Users\28079083\AppData\Local\Temp\ipykernel_14288\1607361849.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fte.rename(columns={'LegalEntity': 'PAID', 'PositionName_EN':'positionName',
C:\Users\28079083\AppData\Local\Temp\ipykernel_14288\1607361849.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fte['used_region_flag'] = fte.apply(lambda x: define_used_region(x), axis=1)


In [223]:
fte_used_columns = ['ID', 'Age', 'Gender', 'PAID', 'MacroEntity', 'positionName', 'OnboardDate', 'duration', 
                    'current_year', 'is_fte', 'current_band', 'on_duty_days']
fte = fte[fte_used_columns]

In [9]:
final_fte = get_one_fte_snapshot(fte)
for i, year in enumerate([2022, 2021, 2020, 2019]):
    fte2022 = fte.copy()
    fte2022['current_year'] = f'{year}-12-31'
    fte2022.query("OnboardDate <= current_year")
    fte2022['on_duty_days'] = (pd.to_datetime(fte2022['current_year']) - fte['OnboardDate']).dt.days
    fte2022['duration'] = (pd.to_datetime('2023-10-31') - fte['OnboardDate']).dt.days    
    fte2022['Age'] = fte2022['Age'] - i - 1
    fte2022 = get_one_fte_snapshot(fte2022)
    final_fte = pd.concat([final_fte, fte2022])
final_fte['src_file'] = 'fte'

#### 合并离职员工和在职员工的训练数据

In [340]:
data_combined = pd.concat([final_fte, final_turnover_data])
data_combined.drop('LastWorkingdate', axis=1, inplace=True)    # 正样本955， 负样本97
train_data = add_more_features(data_combined)
# train_data.to_excel('step_data/train_data.xlsx', index=False)
train_data.to_excel('step_data/train_data.xlsx', index=False)

C:\Users\28079083\AppData\Local\Temp\ipykernel_14288\2696626184.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_org_data.drop(set(org_data_use_columns) - set({'ID'}), axis=1, inplace=True)


In [ ]:
train_data.isnull().sum()

In [432]:
# data_combined.groupby('is_fte').agg({'ID': pd.Series.nunique})   # 30%的离职率

#### 在职员工的预测数据

In [303]:
forecast_fte = fte.copy()
forecast_fte['current_year'] = '2023-10-31'
forecast_fte['on_duty_days'] = (pd.to_datetime('2023-10-31') - forecast_fte['OnboardDate']).dt.days
forecast_fte = get_one_fte_snapshot(forecast_fte)
forecast_fte = add_more_features_forecast(forecast_fte)
forecast_fte.to_excel('model_data/forecast_fte.xlsx', index=False)

In [82]:
# train_data[train_data['Job Family'].isnull()].drop_duplicates(subset=['ID']).to_excel('data/check/org_data_check.xlsx', index=False)

### modeling

In [341]:
model_used_features = pd.read_excel('模型标签.xlsx')['column'].tolist()
categorical_features = ['Gender',  'end_of_date_position','end_of_date_MacroEntity','end_of_date_paid',
                        'Organizational Unit','Line Manager ID', 'Job Family', '员工子组(OM)', '工作地 ID(OM)','Functional Area']

In [342]:
def get_modeling_train_dataset(train_data, save_train=False):
    train_data2 = train_data[(train_data['recent_cr'].notnull())]
    print("Initial turnover records ratio: {:.2%}".format(train_data2['is_fte'].value_counts()[0] / train_data2['is_fte'].value_counts().sum()))   
    train_fte_trunc = train_data.query('is_fte == 1').sample(n=400, random_state=56)            # 让train data的离职人占比20%
    train_data_sub = pd.concat([train_fte_trunc, train_data.query('is_fte == 0')])
    train_data_sub = train_data_sub.reset_index(drop=True)
    train_data_sub = train_data_sub[model_used_features]
    train_data_sub['y'] = 1 - train_data_sub['is_fte']    # y是半年后是否离职，还有一种是y=duration
    if save_train:
        train_data_sub.to_excel("model_data/train.xlsx")
    for col in categorical_features:
        train_data_sub[col] = train_data_sub[col].astype(str)
        train_data_sub[col] = train_data_sub[col].fillna('NA')
    return train_data_sub

In [343]:
train_data_sub = get_modeling_train_dataset(train_data, save_train=False)
X = train_data_sub.drop(['ID', 'duration', 'is_fte', 'y'], axis=1)
y = train_data_sub['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=65)
print("train turnover ratio: {:.2%}".format(y_train.value_counts()[1] / y_train.value_counts().sum()))
print("test turnover ratio: {:.2%}".format(y_test.value_counts()[1] / y_test.value_counts().sum()))

Initial turnover records ratio: 8.36%
train turnover ratio: 17.27%
test turnover ratio: 18.56%


In [344]:
model = CatBoostClassifier(**params)
model.fit(X_train, y_train,eval_set=[(X_train, y_train),(X_test,y_test)],cat_features=categorical_features)
test_pred = [pred[1] for pred in  model.predict_proba(X_test)]
test_auc= roc_auc_score(list(y_test), test_pred)
test_auc

0.9064697609001406

In [345]:
threshold = 0.3
confusion_mat = confusion_matrix(y_test.tolist(), [int(x > threshold) for x in test_pred])
confusionMatrix = pd.DataFrame(confusion_mat, columns=[0, 1], index=[0, 1]). \
    sort_index(axis=0, ascending=False).sort_index(axis=1, ascending=False)
confusionMatrix["tpr"] = confusionMatrix[1] / confusionMatrix.sum(axis=1)
confusionMatrix["fpr"] = confusionMatrix[0] / confusionMatrix.sum(axis=1)
confusionMatrix

,1,0,tpr,fpr
1,15,3,0.833333,0.159292
0,14,65,0.177215,0.820943


In [90]:
params={
    'loss_function': 'Logloss', # 损失函数，取值RMSE, Logloss, MAE, CrossEntropy, Quantile, LogLinQuantile, Multiclass, MultiClassOneVsAll, MAPE, Poisson。默认Logloss。
    'custom_loss': 'AUC', # 训练过程中计算显示的损失函数，取值Logloss、CrossEntropy、Precision、Recall、F、F1、BalancedAccuracy、AUC等等
    'eval_metric': 'AUC', # 用于过度拟合检测和最佳模型选择的指标，取值范围同custom_loss
    'iterations': 40, # 最大迭代次数，默认500. 别名：num_boost_round, n_estimators, num_trees
    'learning_rate': 0.3, # 学习速率,默认0.03 别名：eta
    'random_seed': 23, # 训练的随机种子，别名：random_state
#     'l2_leaf_reg': 5, # l2正则项，别名：reg_lambda
    'bootstrap_type': 'Bernoulli', # 确定抽样时的样本权重，取值Bayesian、Bernoulli(伯努利实验)、MVS(仅支持cpu)、Poisson(仅支持gpu)、No（取值为No时，每棵树为简单随机抽样）;默认值GPU下为Bayesian、CPU下为MVS
#     'bagging_temperature': 0,  # bootstrap_type=Bayesian时使用,取值为1时采样权重服从指数分布；取值为0时所有采样权重均等于1。取值范围[0，inf)，值越大、bagging就越激进
#     'subsample': 0.6, # 样本采样比率（行采样）
    'sampling_frequency': 'PerTreeLevel', # 采样频率，取值PerTree（在构建每棵新树之前采样）、PerTreeLevel（默认值，在子树的每次分裂之前采样）；仅支持CPU
    'use_best_model': True, # 让模型使用效果最优的子树棵树/迭代次数，使用验证集的最优效果对应的迭代次数（eval_metric：评估指标，eval_set：验证集数据），布尔类型可取值0，1（取1时要求设置验证集数据）
#     'best_model_min_trees': 50, # 最少子树棵树,和use_best_model一起使用
    'depth': 4, # 树深，默认值6
    'grow_policy': 'SymmetricTree', # 子树生长策略，取值SymmetricTree（默认值，对称树）、Depthwise（整层生长，同xgb）、Lossguide（叶子结点生长，同lgb）
    'min_data_in_leaf': 6, # 叶子结点最小样本量
#     'max_leaves': 12, # 最大叶子结点数量
#     'one_hot_max_size': 4, # 对唯一值数量<one_hot_max_size的类别型特征使用one-hot编码
#     'rsm': 0.6, # 列采样比率，别名colsample_bylevel 取值（0，1],默认值1
    'nan_mode': 'Min', # 缺失值处理方法，取值Forbidden（不支持缺失值，输入包含缺失时会报错）、Min（处理为该列的最小值，比原最小值更小）、Max（同理）
    'input_borders': None, # 特征数据边界（最大最小边界）、会影响缺失值的处理（nan_mode取值Min、Max时），默认值None、在训练时特征取值的最大最小值即为特征值边界
    'boosting_type': 'Ordered', # 提升类型，取值Ordered（catboost特有的排序提升，在小数据集上效果可能更好，但是运行速度较慢）、Plain（经典提升）
#     'max_ctr_complexity': 2, # 分类特征交叉的最高阶数，默认值4
    'logging_level':'Silent', # 模型训练过程的信息输出等级，取值Silent（不输出信息）、Verbose（默认值，输出评估指标、已训练时间、剩余时间等）、Info（输出额外信息、树的棵树）、Debug（debug信息）
    'metric_period': 1, # 计算目标值、评估指标的频率，默认值1、即每次迭代都输出目标值、评估指标
#     'early_stopping_rounds': 20,
#     'border_count': 254, # 数值型特征的分箱数，别名max_bin，取值范围[1,65535]、默认值254（CPU下), # 设置提前停止训练，在得到最佳的评估结果后、再迭代n（参数值为n）次停止训练，默认值不启用
#     'feature_border_type': 'GreedyLogSum', # 数值型特征的分箱方法，取值Median、Uniform、UniformAndQuantiles、MaxLogSum、MinEntropy、GreedyLogSum（默认值）
}

In [87]:
model = CatBoostClassifier(**params)
model.fit(X_train, y_train,eval_set=[(X_train, y_train),(X_test,y_test)],cat_features=categorical_features)
test_pred = [pred[1] for pred in  model.predict_proba(X_test)]
test_auc= roc_auc_score(list(y_test), test_pred)
test_auc

In [210]:
# model result
result_save_file = pd.ExcelWriter("model_data/model_result.xlsx")
feature_importance = pd.Series(model.feature_importances_, index=X_train.columns, name="feature_importance")
feature_importance.sort_values(ascending=False)
feature_importance.to_excel(result_save_file, sheet_name='feature_importance')

threshold = 0.3
confusion_mat = confusion_matrix(y_test.tolist(), [int(x > threshold) for x in test_pred])
confusionMatrix = pd.DataFrame(confusion_mat, columns=[0, 1], index=[0, 1]). \
    sort_index(axis=0, ascending=False).sort_index(axis=1, ascending=False)
confusionMatrix["tpr"] = confusionMatrix[1]/confusionMatrix.sum(axis=1)
confusionMatrix["fpr"] = confusionMatrix[0]/confusionMatrix.sum(axis=1)
confusionMatrix.to_excel(result_save_file, sheet_name='confusion_matrix')
result_save_file.close()

with open("model_data/catboost_model.pk", "wb") as f:
    pickle.dump(model, f)
with open("model_data/catboost_model_params.txt", 'w') as f:
    json.dump(params, f)

### 预测

In [304]:
forecast_data = forecast_fte[X_train.columns]
for col in categorical_features:
    forecast_data[col] = forecast_data[col].astype(str)
    forecast_data[col] = forecast_data[col].fillna('NA')
    
predict_proba = [pred[1] for pred in  model.predict_proba(forecast_data)]
forecast_result = pd.concat([forecast_fte, pd.Series(predict_proba, name='predict_p', index=forecast_fte.index)], axis=1)
forecast_result['turnover_tag'] = forecast_result['predict_p'].apply(lambda x: 1 if x > threshold else 0)
forecast_result['turnover_tag'].value_counts()
forecast_result.to_excel('model_data/forecast_result2023.xlsx', index=False)

C:\Users\28079083\AppData\Local\Temp\ipykernel_14288\449253622.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data[col] = forecast_data[col].astype(str)
C:\Users\28079083\AppData\Local\Temp\ipykernel_14288\449253622.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_data[col] = forecast_data[col].fillna('NA')


In [116]:
threshold = 0.33
confusion_mat = confusion_matrix(y_test.tolist(), [int(x > threshold) for x in test_pred])
confusionMatrix = pd.DataFrame(confusion_mat, columns=[0, 1], index=[0, 1]). \
    sort_index(axis=0, ascending=False).sort_index(axis=1, ascending=False)
confusionMatrix["tpr"] = confusionMatrix[1] / confusionMatrix.sum(axis=1)
confusionMatrix["fpr"] = confusionMatrix[0] / confusionMatrix.sum(axis=1)